In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.1 MB/s eta 0:00:00


In [4]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import time


from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error
from tqdm.auto import tqdm

import warnings
warnings.simplefilter("ignore")

import unicodedata
import lightgbm as lgb

import optuna

In [5]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_trials = 3000
    #save_dir = "G:/マイドライブ/signate_StudentCup2023/exp/"
    #filename = "exp013"

In [6]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [11]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')

In [13]:
# ===================================================================
#  DataLoading
# ===================================================================
df_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp00038/signate-models/exp38_oof_pred.csv').rename(columns={"oof_pred":"pred_1"})
df_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp00039/signate-models/exp39_oof_pred.csv').rename(columns={"oof_pred":"pred_2"})

In [18]:
df = pd.concat([df_1, df_2, df_train['price']], axis=1)

In [19]:
# ===================================================================
#  optuna
# ===================================================================
def objective(trial):
    a = trial.suggest_float("a", 0, 1)
    b = trial.suggest_float("b", 0, 1)

    df[f"pred"] = df[f"pred_1"] * a + df[f"pred_2"] * b
    score = get_score(y_true=df["price"], y_pred = df[f"pred"])
    return score

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize",
                            sampler=optuna.samplers.TPESampler(seed=CFG.seed))
study.optimize(objective,
                n_trials=CFG.n_trials,
                show_progress_bar=True)

best_params = study.best_params
best_value = study.best_value
print(best_params, best_value)

  0%|          | 0/3000 [00:00<?, ?it/s]

{'a': 0.7641994986787857, 'b': 0.24524302335892614} 44.01313802461185


In [23]:
# ===================================================================
#  test
# ===================================================================
test_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp00038/signate-models/submission.csv', header=None).rename(columns={0:"id", 1:"pred_1"})
test_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp00039/signate-models/submission.csv', header=None).rename(columns={0:"id", 1:"pred_2"})

test_1["pred"] = test_1["pred_1"] * best_params["a"] + test_2["pred_2"] * best_params["b"]
test_1[["id", "pred"]].to_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp38_39_submission.csv', index=False, header=None)
test_1[["id", "pred"]].head(2)

,id,pred
0,27532,9589.293003
1,27533,5519.364689
